# Technical Report
#### Collegiate Courses - New Textbook Library and Bookstore Data

### Overview
* We're taking raw data, extracting the information we want, transforming the information so it is in a universal format, and loading into a more user friendly database so analysts can more easily read and present the data. We set a goal of providing this data in ((2nd normal form)) for our analysts.

### Extract
* We downloaded two files, one with textbook data, and another with college course data.

* The first thing we did was use pandas to remove null values, and information that was not relevent to our requirements, such as "digital books", or isolating the "spring 2022" courses, etc.


### Transform
* We reformatted some numeric data to remove dollar signs, making them more user friendly as float type integers

# In pandas database manipulation tools, .toframe and .random.42 we also explored our data to look for formatting would be non-functional

* We selected only certain columns to load: (or selecting null columns not to load). For example, columns with null values, or irrelevant for analysis, or redundant.

* We encoded free-form values in term column to 'SPRING 2022', and normalized all text type to caps in all data fields, all snake_case in header fields. 

* Looking up and validating the relevant data from tables or referential files
* We merged data from the courses table & textbooks table on columns `department_id` & `course_id` using pandas 

* In the merged table we reduced the list that would be interesting to student consumers or bookstore owners in a way that makes it easy for analysts to display as they choose.

### Load
* We used PostgreSQL to upload our data into a database called `library_db`

* We then created file `populate.sql` to create a tables called `courses`, `textbooks`, & `course_textbook` and used PostgresSQl Import tool to populate the data from the csv files of the same name.
